In [1]:
###Procedure to define neural network

In [2]:
#Define the neural network that has some learnable parameters (Weights)
#Iterate over a dataset of inputs
#Process input through network
#Compute the loss (How far output from being correct)
#Propagate gradients back into the network's parameter
#Updates the weights of the NN
## Weight = weight - learing_rate * gradient

In [3]:
###Define the nn

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class Net (nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        #1 i/p image channel,6 o/p channel, 3x3 square convol
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation; y =Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120) #6*6 img dime
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        
        #Max pool over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        #If the size is a squre u can specify single no
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    
    def num_flat_features(self, x):
        size = x.size()[1:] #all the dim except batch dim
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
 
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [6]:
params = list(net.parameters())

print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [7]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0903, -0.0094, -0.0606,  0.0255, -0.1025, -0.0963,  0.0381, -0.0730,
         -0.0544,  0.0054]], grad_fn=<AddmmBackward>)


In [8]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [9]:
#Loss function

In [10]:
output = net(input)
target = torch.randn(10) #dummy targer for ex
target = target.view(1, -1) #make is same shape as o/p
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(3.0167, grad_fn=<MseLossBackward>)


In [11]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0]) #linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #relu

In [12]:
#Backpropagation

In [13]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


In [14]:
loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([-0.0642,  0.0087,  0.0022,  0.0082,  0.0154,  0.0318])


In [15]:
#Update the weights

# weight = weight - learning_rate * gradient

In [16]:
#sgd

learning_rate = 0.01

for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [17]:
import torch.optim as optim

# creating optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your trainning loop:
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # does the update
